# LM Format Enforcer Integration with vLLM

<a target="_blank" href="https://colab.research.google.com/github/noamgat/lm-format-enforcer/blob/main/samples/colab_vllm_integration.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook shows how you can integrate with the vLLM library. vLLM does not currently have an API for token filtering, so we have to do some monkey patching to expose the functionality.

## Setting up the COLAB runtime (user action required)

This colab-friendly notebook is targeted at demoing the enforcer on LLAMA2. It can run on a free GPU on Google Colab.
Make sure that your runtime is set to GPU:

Menu Bar -> Runtime -> Change runtime type -> T4 GPU (at the time of writing this notebook). [Guide here](https://www.codesansar.com/deep-learning/using-free-gpu-tpu-google-colab.htm).

## Gathering huggingface credentials (user action required)

We begin by installing the dependencies. This demo uses llama2, so you will have to create a free huggingface account, request access to the llama2 model, create an access token, and insert it when executing the next cell will request it.

Links:

- [Request access to llama model](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf). See the "Access Llama 2 on Hugging Face" section.
- [Create huggingface access token](https://huggingface.co/settings/tokens)


In [1]:
!pip install vllm lm-format-enforcer
!huggingface-cli login

# When running from source / developing the library, use this instead
# %load_ext autoreload
# %autoreload 2
# import sys
# import os
# sys.path.append(os.path.abspath('..'))
## os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

## Creating a custom sampler that filters tokens

We introduce a subclass of vLLM's ```SamplingParams``` that also accepts a token filtering function, with the same API as Huggingface Transformers 

```prefix_allowed_tokens_fn: Callable[[int, torch.Tensor], List[int]]```

We then introduce the function ```_apply_allowed_token_filters()``` that applies the filter functions to the logits (sets them to negative infinity if not allowed) to requests that contain a filter function.

We hope that in future releases of vLLM, this (or similar) will be part of vLLM's ```Sampler``` class.

In [2]:
import vllm
import torch
from typing import List, Callable, Optional
from vllm.sampling_params import SamplingParams
from vllm.model_executor.input_metadata import InputMetadata

class SamplingParamsWithFilterFunction(SamplingParams):
    logits_allowed_tokens_filter_function: Optional[Callable[[int, torch.Tensor], List[int]]]

def _apply_allowed_token_filters(logits: torch.Tensor, 
                                 input_metadata: InputMetadata) -> torch.Tensor:
    num_seqs, vocab_size = logits.shape
    logits_row_idx = 0
    for seq_ids, sampling_params in input_metadata.seq_groups:
        if isinstance(sampling_params, SamplingParamsWithFilterFunction):
            filter_function = sampling_params.logits_allowed_tokens_filter_function
        else:
            filter_function = None
        for seq_id in seq_ids:
            if filter_function is not None:
                output_token_ids = input_metadata.seq_data[seq_id].output_token_ids
                output_token_tensor = torch.tensor(output_token_ids, dtype=torch.long)
                allowed_tokens = filter_function(logits_row_idx, output_token_tensor)
                logits_add_factor = torch.zeros(vocab_size, dtype=logits.dtype, device=logits.device)
                logits_add_factor[:] = float('-inf')
                logits_add_factor[allowed_tokens] = 0
                logits[logits_row_idx] += logits_add_factor
            logits_row_idx += 1
    assert logits_row_idx == num_seqs
    return logits 



/home/noamgat/mambaforge/envs/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-18 21:01:08,225	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-10-18 21:01:08,604	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In order to integrate this function with the ```Sampler``` class, we have to change its ```forward()``` function to call it. Since we are not modifying vLLM itself, we will do this with monkey patching. 

Other than the line 
```
logits = _apply_allowed_token_filters(logits, input_metadata)
```
this is a 100% copy of the original ```Sampler.forward()``` function.

In [3]:
from vllm.model_executor.layers.sampler import SamplerOutput, _prune_hidden_states, _get_logits, _get_output_tokens, _get_penalties, _apply_penalties, _get_temperatures, _get_top_p_top_k, _apply_top_p_top_k, _sample, _get_logprobs, _build_sampler_output, _SAMPLING_EPS

from typing import Optional

# @wrapt.patch_function_wrapper(vllm.model_executor.layers.Sampler, 'forward')
def patched_forward(
        self,
        embedding: torch.Tensor,
        hidden_states: torch.Tensor,
        input_metadata: InputMetadata,
        embedding_bias: Optional[torch.Tensor] = None,
    ) -> SamplerOutput:
        # Get the hidden states that we use for sampling.
        hidden_states = _prune_hidden_states(hidden_states, input_metadata)

        # Get the logits for the next tokens.
        logits = _get_logits(hidden_states, embedding, embedding_bias,
                             self.vocab_size)
        
        # Apply presence and frequency penalties.
        output_tokens = _get_output_tokens(input_metadata)
        assert len(output_tokens) == logits.shape[0]
        presence_penalties, frequency_penalties = _get_penalties(
            input_metadata)
        assert len(presence_penalties) == logits.shape[0]
        assert len(frequency_penalties) == logits.shape[0]
        logits = _apply_penalties(logits, output_tokens, presence_penalties,
                                  frequency_penalties)
        
        ### LM FORMAT ENFORCER MONKEY PATCH START
        logits = _apply_allowed_token_filters(logits, input_metadata)
        ### LM FORMAT ENFORCER MONKEY PATCH END

        # Apply temperature scaling.
        temperatures = _get_temperatures(input_metadata)
        assert len(temperatures) == logits.shape[0]
        if any(t != 1.0 for t in temperatures):
            t = torch.tensor(temperatures,
                             dtype=logits.dtype,
                             device=logits.device)
            # Use in-place division to avoid creating a new tensor.
            logits.div_(t.unsqueeze(dim=1))

        # Apply top-p and top-k truncation.
        top_ps, top_ks = _get_top_p_top_k(input_metadata, self.vocab_size)
        assert len(top_ps) == len(top_ks) == logits.shape[0]
        do_top_p = any(p < 1.0 - _SAMPLING_EPS for p in top_ps)
        do_top_k = any(k != self.vocab_size for k in top_ks)
        if do_top_p or do_top_k:
            logits = _apply_top_p_top_k(logits, top_ps, top_ks)

        # We use float32 for probabilities and log probabilities.
        # Compute the probabilities.
        probs = torch.softmax(logits, dim=-1, dtype=torch.float)
        # Compute the log probabilities.
        # Use log_softmax to ensure numerical stability.
        logprobs = torch.log_softmax(logits, dim=-1, dtype=torch.float)

        # Sample the next tokens.
        sample_results = _sample(probs, logprobs, input_metadata)
        # Get the logprobs query results.
        prompt_logprobs, sample_logprobs = _get_logprobs(
            logprobs, input_metadata, sample_results)
        return _build_sampler_output(sample_results, input_metadata,
                                     prompt_logprobs, sample_logprobs)

We load the model, as is normally done with vLLM

In [4]:
model_id = 'meta-llama/Llama-2-7b-chat-hf'
llm = vllm.LLM(model=model_id)

INFO 10-18 21:01:10 llm_engine.py:72] Initializing an LLM engine with config: model='meta-llama/Llama-2-7b-chat-hf', tokenizer='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)
INFO 10-18 21:01:10 tokenizer.py:31] For some LLaMA V1 models, initializing the fast tokenizer may take a long time. To reduce the initialization time, consider using 'hf-internal-testing/llama-tokenizer' instead of the original tokenizer.
INFO 10-18 21:07:48 llm_engine.py:207] # GPU blocks: 1091, # CPU blocks: 512
WARNING 10-18 21:07:48 cache_engine.py:96] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.


If the previous cell executed successfully, you have propertly set up your Colab runtime and huggingface account!

A few helper functions to make display nicer.

In [5]:
from IPython.display import display, Markdown

def display_header(text):
    display(Markdown(f'**{text}**'))

def display_content(text):
    display(Markdown(f'```\n{text}\n```'))

## Setting up the prompt for the specific language model

We set up the prompting style according to the demo at https://huggingface.co/spaces/huggingface-projects/llama-2-13b-chat/blob/main/app.py . We simplify the implementation a bit as we don't need chat history for this demo.

In [6]:
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\
"""

def get_prompt(message: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    texts = [f'<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n']
    # The first user input is _not_ stripped
    do_strip = False
    message = message.strip() if do_strip else message
    texts.append(f'{message} [/INST]')
    return ''.join(texts)

## Activating the monkey patch and creating the generation function

We monkey-patch the ```Sampler``` class with our custom ```forward()``` method, using ```unittest.mock```.

We use our sampling params in order to sent the specific filter function with the request. Different requests can have different format enforcers.

In [7]:
from lmformatenforcer import build_transformers_prefix_allowed_tokens_fn, CharacterLevelParser
from unittest import mock

DEFAULT_MAX_NEW_TOKENS = 100

def vllm_with_character_level_parser(llm: vllm.LLM, prompt: str, parser: Optional[CharacterLevelParser] = None) -> str:
    with mock.patch.object(vllm.model_executor.layers.sampler.Sampler, 'forward', patched_forward):
        prefix_function = build_transformers_prefix_allowed_tokens_fn(llm.get_tokenizer(), parser) if parser else None
        sampling_params = SamplingParamsWithFilterFunction()
        sampling_params.max_tokens = DEFAULT_MAX_NEW_TOKENS
        sampling_params.logits_allowed_tokens_filter_function = prefix_function
        result = llm.generate(prompt, sampling_params=sampling_params)
        return result[0].outputs[0].text

## vLLM + JSON Use case

Now we demonstrate using ```JsonSchemaParser```. We create a pydantic model, generate the schema from it, and use that to enforce the format.
The output will always be in a format that can be parsed by the parser.

In [8]:
from lmformatenforcer import JsonSchemaParser
from pydantic import BaseModel

from typing import List

class AnswerFormat(BaseModel):
    first_name: str
    last_name: str
    year_of_birth: int
    num_seasons_in_nba: int

question = 'Please give me information about Michael Jordan. You MUST answer using the following json schema: '
question_with_schema = f'{question}{AnswerFormat.schema_json()}'
prompt = get_prompt(question_with_schema)

display_header("Prompt:")
display_content(prompt)

display_header("Answer, With json schema enforcing:")

result = vllm_with_character_level_parser(llm, prompt, JsonSchemaParser(AnswerFormat.schema()))
display_content(result)

display_header("Answer, Without json schema enforcing:")
result = vllm_with_character_level_parser(llm, prompt, None)
display_content(result)



**Prompt:**

```
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Please give me information about Michael Jordan. You MUST answer using the following json schema: {"title": "AnswerFormat", "type": "object", "properties": {"first_name": {"title": "First Name", "type": "string"}, "last_name": {"title": "Last Name", "type": "string"}, "year_of_birth": {"title": "Year Of Birth", "type": "integer"}, "num_seasons_in_nba": {"title": "Num Seasons In Nba", "type": "integer"}}, "required": ["first_name", "last_name", "year_of_birth", "num_seasons_in_nba"]} [/INST]
```

**Answer, With json schema enforcing:**

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.32s/it]

```
  { "first_name": "Michael", "last_name": "Jordan", "year_of_birth": 1963, "num_seasons_in_nba": 15 }




```

**Answer, Without json schema enforcing:**

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.12s/it]


```
  Of course, I'd be happy to provide you with information about Michael Jordan! Here is the information in the format you requested:
{
"title": "AnswerFormat",
"type": "object",
"properties": {
"first_name": {
"title": "First Name",
"type": "string",
"description": "Michael Jordan's first name."
},
"last_name": {

"title": "Last
```

As you can see, the enforced output matches the required schema, while the unenforced does not. We have successfully integrated with vLLM!